In [2]:
from __future__ import annotations


import re

import z3
from sympy import Matrix
from sympy import solve_linear_system
from sympy import symbols
import string

In [3]:
line = "[.###.#..] (0,2,3,4,6,7) (1,2,4,5,6) (1,3,4,5,7) (0,2,4) (4,7) (0,1,4,6) (1,2,3,4,5,7) (2,4,7) {29,44,42,26,66,27,30,36}"
m = re.match(r"\[(.*?)\]\s*(.*?)\s*\{(.*?)\}", line)
if not m:
    raise ValueError("Input string does not match expected format")

lights, buttons, joltages = m.groups()
lights = int(
    lights.replace(".", "0").replace("#", "1")[::-1], 2
)  # bitmap, invert for the bit indexes to match
buttons = re.findall(r"\(([^)]*)\)", buttons)
buttons = [{int(bb) for bb in b.strip().split(",")} for b in buttons]
joltages = [int(j) for j in joltages.split(",")]
row = (lights, buttons, joltages)

In [ ]:
b = Matrix(joltages)

n_rows = max(max(s) for s in buttons) + 1
n_cols = len(buttons)
A = Matrix.zeros(n_rows, n_cols)
for col, rows in enumerate(buttons):
    for row in rows:
        A[row, col] = 1

A_b = A.row_join(b)
A_b

Matrix([
[1, 0, 0, 1, 0, 1, 0, 0, 29],
[0, 1, 1, 0, 0, 1, 1, 0, 44],
[1, 1, 0, 1, 0, 0, 1, 1, 42],
[1, 0, 1, 0, 0, 0, 1, 0, 26],
[1, 1, 1, 1, 1, 1, 1, 1, 66],
[0, 1, 1, 0, 0, 0, 1, 0, 27],
[1, 1, 0, 0, 0, 1, 0, 0, 30],
[1, 0, 1, 0, 1, 0, 1, 1, 36]])

In [5]:
n_unknowns = A_b.shape[-1] - 1

vars_all = symbols(" ".join(string.ascii_lowercase[-n_unknowns:]))
vars_all

(s, t, u, v, w, x, y, z)

In [6]:
sol = solve_linear_system(A_b, *vars_all)
sol

{s: 6, t: 7, u: z - 3, v: 6, w: 10 - z, x: 17, y: 23 - z}

In [7]:
solved_vars = set(sol.keys())
free_vars = [vv for vv in vars_all if vv not in solved_vars]
free_vars

[z]

In [8]:
X = Matrix(vars_all)
X_sol = X.subs(sol)
X_sol

Matrix([
[     6],
[     7],
[ z - 3],
[     6],
[10 - z],
[    17],
[23 - z],
[     z]])

In [9]:
X_particular = X_sol.subs({var: 0 for var in free_vars})
X_particular

Matrix([
[ 6],
[ 7],
[-3],
[ 6],
[10],
[17],
[23],
[ 0]])

In [ ]:
all_vectors = {}
for idx, var in enumerate(free_vars):
    d_var = X_sol.subs({vv: 1 if vv == var else 0 for vv in free_vars}) - X_particular
    all_vectors[f"k_{idx}"] = d_var

In [ ]:
len(all_vectors)

1

# Z3

In [ ]:
scale_factors_k = {}
for name in all_vectors:
    scale_factors_k[name] = z3.Int(name)  # TODO: Nicer

vector_x = []
for i in range(X_particular.shape[0]):
    terms = [int(X_particular[i])]
    for k_name in all_vectors:
        fac, vec = scale_factors_k[k_name], all_vectors[k_name]
        terms.append(fac * int(vec[i]))
    x_row = sum(terms)
    vector_x.append(x_row)

print(vector_x)

optim = z3.Optimize()
optim.minimize(sum(vector_x))

for xxx in vector_x:
    optim.add(xxx >= 0)

for kkk in scale_factors_k.values():
    optim.add(kkk >= 0)

assert optim.check()
M = optim.model()
M

[6 + k_0*0, 7 + k_0*0, -3 + k_0*1, 6 + k_0*0, 10 + k_0*-1, 17 + k_0*0, 23 + k_0*-1, 0 + k_0*1]


[k_0 = 3]

In [20]:
vector_x

[6 + k_0*0,
 7 + k_0*0,
 -3 + k_0*1,
 6 + k_0*0,
 10 + k_0*-1,
 17 + k_0*0,
 23 + k_0*-1,
 0 + k_0*1]

In [21]:
free_vars_z3 = M.decls()
sol_values = {str(d): M[d].as_long() for d in free_vars_z3}

In [22]:
sol_values

{'k_0': 3}

In [23]:
X = X_particular
for k, v in all_vectors.items():
    X += sol_values[k] * v
sum(X)

66

In [24]:
assert A * X == b
A * X

Matrix([
[29],
[44],
[42],
[26],
[66],
[27],
[30],
[36]])

In [25]:
X

Matrix([
[ 6],
[ 7],
[ 0],
[ 6],
[ 7],
[17],
[20],
[ 3]])